In [1]:
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

class CustomEfficientNet(nn.Module):
    def __init__(self, model_name="efficientnet-b0", num_classes=6, freeze_backbone=False):
        super(CustomEfficientNet, self).__init__()
        # 사전학습된 EfficientNet 불러오기
        self.efficientnet = EfficientNet.from_pretrained(model_name)
        
        # (옵션) 일부 레이어 동결
        if freeze_backbone:
            for param in self.efficientnet.parameters():
                param.requires_grad = False
        
        # EfficientNet에서 마지막 FC의 입력 차원 가져오기
        in_features = self.efficientnet._fc.in_features
        
        # 새로운 분류기(FC) 레이어 정의
        # 원하는 대로 Linear + Dropout + BatchNorm 등을 쌓을 수 있음
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3),                  # Dropout 예시
            nn.Linear(in_features, 256),        
            nn.ReLU(),
            nn.BatchNorm1d(256),                # BatchNorm1d 예시
            nn.Dropout(p=0.3),                  # 추가 Dropout
            nn.Linear(256, num_classes)         # 최종 출력층
        )
        
        # 기존 EfficientNet의 분류 레이어를 우리가 정의한 classifier로 교체
        # (EfficientNet이 _fc 속성을 통해 분류를 담당함)
        self.efficientnet._fc = self.classifier

    def forward(self, x):
        return self.efficientnet(x)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# ===============================
# 1. 하이퍼파라미터 및 세팅
# ===============================
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 128
epochs = 128
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===============================
# 2. 데이터셋 & 전처리
# ===============================
# 예: 데이터 증강을 좀 더 다양하게 사용 가능
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),    # 수평 뒤집기
    transforms.RandomRotation(degrees=15),     # 회전
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"
# 전체 데이터셋 (train_transform를 먼저 적용)
full_dataset = datasets.ImageFolder(root=dataset_path, transform=train_transform)

# 80%: Train / 20%: Test 로 분할
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# 검증 세트에는 test_transform 적용을 위해 transform 교체
test_dataset.dataset.transform = test_transform

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

# ===============================
# 3. 모델 초기화 (커스텀 모델)
# ===============================
#from custom_efficientnet import CustomEfficientNet  # (위 클래스가 들어있다고 가정)
model = CustomEfficientNet(model_name=model_name,
                           num_classes=num_classes,
                           freeze_backbone=False).to(device)

# ===============================
# 4. 손실 함수 & 최적화 함수
# ===============================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ===============================
# 5. 학습 함수
# ===============================
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# ===============================
# 6. 평가 함수
# ===============================
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# ===============================
# 7. 학습 루프
# ===============================
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test  Loss: {test_loss:.4f} | Test  Acc: {test_acc:.4f}")

print("Training complete!")


Loaded pretrained weights for efficientnet-b0
Epoch [1/20]
  Train Loss: 1.2695 | Train Acc: 0.5282
  Test  Loss: 1.3913 | Test  Acc: 0.4407
Epoch [2/20]
  Train Loss: 0.4020 | Train Acc: 0.8997
  Test  Loss: 1.0705 | Test  Acc: 0.5706
Epoch [3/20]
  Train Loss: 0.1860 | Train Acc: 0.9619
  Test  Loss: 0.7950 | Test  Acc: 0.7175
Epoch [4/20]
  Train Loss: 0.0770 | Train Acc: 0.9915
  Test  Loss: 0.7790 | Test  Acc: 0.7514
Epoch [5/20]
  Train Loss: 0.0546 | Train Acc: 0.9929
  Test  Loss: 0.8192 | Test  Acc: 0.7627
Epoch [6/20]
  Train Loss: 0.0522 | Train Acc: 0.9915
  Test  Loss: 0.8735 | Test  Acc: 0.7684
Epoch [7/20]
  Train Loss: 0.0414 | Train Acc: 0.9929
  Test  Loss: 0.9158 | Test  Acc: 0.7571
Epoch [8/20]
  Train Loss: 0.0230 | Train Acc: 0.9929
  Test  Loss: 0.9084 | Test  Acc: 0.7797
Epoch [9/20]
  Train Loss: 0.0166 | Train Acc: 0.9944
  Test  Loss: 0.8115 | Test  Acc: 0.8023
Epoch [10/20]
  Train Loss: 0.0197 | Train Acc: 0.9929
  Test  Loss: 0.7005 | Test  Acc: 0.8362
Epo

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# CustomEfficientNet 클래스 정의
class CustomEfficientNet(nn.Module):
    def __init__(self, model_name="efficientnet-b0", num_classes=6, freeze_backbone=False):
        super(CustomEfficientNet, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained(model_name)
        if freeze_backbone:
            for param in self.efficientnet.parameters():
                param.requires_grad = False
        
        in_features = self.efficientnet._fc.in_features
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(p=0.3),
            nn.Linear(256, num_classes)
        )
        self.efficientnet._fc = self.classifier

    def forward(self, x):
        return self.efficientnet(x)

# 하이퍼파라미터 설정
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 128
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리 및 로드
image_size = EfficientNet.get_image_size(model_name)
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
model = CustomEfficientNet(model_name=model_name, num_classes=num_classes, freeze_backbone=False).to(device)

# 손실 함수 및 최적화기 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

# 평가 함수
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

# 학습 루프
best_test_acc = 0.0  # 최고 Test Accuracy 저장
best_epoch = 0       # 최고 Accuracy를 기록한 Epoch
early_stopping = EarlyStopping(patience=10)  # Early Stopping 설정

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    # 최고 Test Accuracy 갱신
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1  # Epoch 번호 저장

    # Early Stopping 체크
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Epoch 결과 출력
    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test  Loss: {test_loss:.4f}  | Test Acc: {test_acc:.4f}")

# 최고 Test Accuracy 출력
print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b0
Epoch [1/128]
  Train Loss: 1.0410 | Train Acc: 0.6201
  Test  Loss: 1.6658 | Test  Acc: 0.4802
Epoch [2/128]
  Train Loss: 0.4192 | Train Acc: 0.8686
  Test  Loss: 0.8904 | Test  Acc: 0.7062
Epoch [3/128]
  Train Loss: 0.2538 | Train Acc: 0.9237
  Test  Loss: 0.7234 | Test  Acc: 0.7458
Epoch [4/128]
  Train Loss: 0.1541 | Train Acc: 0.9506
  Test  Loss: 0.6977 | Test  Acc: 0.8079
Epoch [5/128]
  Train Loss: 0.2125 | Train Acc: 0.9407
  Test  Loss: 0.5445 | Test  Acc: 0.8418
Epoch [6/128]
  Train Loss: 0.1500 | Train Acc: 0.9576
  Test  Loss: 0.8973 | Test  Acc: 0.7684
Epoch [7/128]
  Train Loss: 0.1295 | Train Acc: 0.9689
  Test  Loss: 0.5580 | Test  Acc: 0.8531
Epoch [8/128]
  Train Loss: 0.1073 | Train Acc: 0.9661
  Test  Loss: 0.6404 | Test  Acc: 0.8475
Epoch [9/128]
  Train Loss: 0.1159 | Train Acc: 0.9647
  Test  Loss: 0.5849 | Test  Acc: 0.8701
Epoch [10/128]
  Train Loss: 0.0691 | Train Acc: 0.9788
  Test  Loss: 0.5354 | Test  Acc: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 모델 클래스 정의 (CustomEfficientNet)
class CustomEfficientNet(nn.Module):
    def __init__(self, model_name="efficientnet-b0", num_classes=6, freeze_backbone=False):
        super(CustomEfficientNet, self).__init__()
        self.backbone = EfficientNet.from_pretrained(model_name)
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False
        in_features = self.backbone._fc.in_features
        self.backbone._fc = nn.Identity()
        self.extra_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_features, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.backbone.extract_features(x)
        x = self.extra_conv(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 하이퍼파라미터
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리 & 로더
image_size = EfficientNet.get_image_size(model_name)
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 및 손실 함수, 최적화기
model = CustomEfficientNet(model_name=model_name, num_classes=num_classes, freeze_backbone=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 및 평가 함수
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, pred = outputs.max(1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, pred = outputs.max(1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    return running_loss/total, correct/total

# 학습 루프
best_test_acc = 0.0
best_epoch = 0
early_stopping = EarlyStopping(patience=10)

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test Loss: {test_loss:.4f}  | Test Acc: {test_acc:.4f}")

print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b0


AttributeError: 'CustomEfficientNet' object has no attribute 'summary'

In [6]:
from torchsummary import summary

# 모델 초기화
model = CustomEfficientNet(model_name=model_name, num_classes=num_classes, freeze_backbone=False).to(device)

# 모델 요약 출력
summary(model, input_size=(3, image_size, image_size))

    
    

Loaded pretrained weights for efficientnet-b0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13        

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 모델 클래스 정의 (CustomEfficientNet)
class CustomEfficientNet(nn.Module):
    def __init__(self, model_name="efficientnet-b0", num_classes=6, freeze_backbone=False):
        super(CustomEfficientNet, self).__init__()
        self.backbone = EfficientNet.from_pretrained(model_name)
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False
        in_features = self.backbone._fc.in_features
        self.backbone._fc = nn.Identity()
        self.extra_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_features, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(), 
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.backbone.extract_features(x)
        x = self.extra_conv(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 하이퍼파라미터
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리 & 로더
image_size = EfficientNet.get_image_size(model_name)

dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 및 손실 함수, 최적화기
model = CustomEfficientNet(model_name=model_name, num_classes=num_classes, freeze_backbone=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 및 평가 함수
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, pred = outputs.max(1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, pred = outputs.max(1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    return running_loss/total, correct/total

# 학습 루프
best_test_acc = 0.0
best_epoch = 0
early_stopping = EarlyStopping(patience=10)

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test Loss: {test_loss:.4f}  | Test Acc: {test_acc:.4f}")

print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")

Loaded pretrained weights for efficientnet-b0
[Epoch 1/50] Train Loss: 1.0384 | Train Acc: 0.6172
  Test Loss: 1.3820  | Test Acc: 0.5706
[Epoch 2/50] Train Loss: 0.4306 | Train Acc: 0.8630
  Test Loss: 1.2499  | Test Acc: 0.7232
[Epoch 3/50] Train Loss: 0.2741 | Train Acc: 0.9153
  Test Loss: 1.6297  | Test Acc: 0.7514
[Epoch 4/50] Train Loss: 0.3404 | Train Acc: 0.8983
  Test Loss: 0.7878  | Test Acc: 0.7571
[Epoch 5/50] Train Loss: 0.2869 | Train Acc: 0.9011
  Test Loss: 0.8492  | Test Acc: 0.8644
[Epoch 6/50] Train Loss: 0.2274 | Train Acc: 0.9308
  Test Loss: 0.9488  | Test Acc: 0.8475
[Epoch 7/50] Train Loss: 0.1981 | Train Acc: 0.9336
  Test Loss: 1.2091  | Test Acc: 0.7797
[Epoch 8/50] Train Loss: 0.2540 | Train Acc: 0.9463
  Test Loss: 0.9144  | Test Acc: 0.8305
[Epoch 9/50] Train Loss: 0.2039 | Train Acc: 0.9576
  Test Loss: 0.9671  | Test Acc: 0.8418
[Epoch 10/50] Train Loss: 0.1771 | Train Acc: 0.9534
  Test Loss: 0.6136  | Test Acc: 0.8588
[Epoch 11/50] Train Loss: 0.1927 